In [1]:
import numpy as np
from implementations import *
from proj1_helpers import *
import matplotlib.pyplot as plt
from costs import *
from hw_helpers import *
from cross_validation import *
from visualization import *
from gradients import*

In [2]:
yb, input_data, ids = load_csv_data('Data/train.csv')

In [3]:
yb_te, input_data_te, ids_te = load_csv_data('Data/test.csv')

In [ ]:
#replace the -999 by nan values
#cop = input_data.copy()
input_data = np.delete(input_data, [4,5,6,12,26,27,28],axis= 1)
np.place(input_data[:29], input_data[:29] is 0, np.NAN)
input_data_te = np.delete(input_data_te, [4,5,6,12,23,24,25,26,27,28],axis= 1)
np.place(input_data[:29], input_data[:29] is 0, np.NAN)
np.place(input_data, input_data < -998, np.NAN)

In [5]:
std_input = standardize(input_data)

In [6]:
std_input = np.nan_to_num(std_input)

#input_data.shape
#np.nanmean(input_data, axis = 0).shape
#np.subtract(input_data, np.nanmean(input_data, axis = 0))

In [7]:
std_input_te = standardize_test(input_data, input_data_te)
std_input_te = np.nan_to_num(std_input_te)

Cross-Validation

In [ ]:
def cross_valid():
    seed = 1
    k_fold = 12
    #lambdas = np.logspace(-4,0,10)
    gamma = np.logspace(-5,-3.7,30)
    k_indices = build_k_indices(yb, k_fold, seed)
    rmse_tr = []
    rmse_te = []
    for ind, gamma_ in enumerate(gamma):
        print(gamma_)
        k_rmse_tr = 0
        k_rmse_te = 0
        for k in range(k_fold):
            print("fold", k)
            loss_tr, loss_te = cross_validation(reg_logistic_regression, yb, std_input, k_indices, k, 0.7,10000, gamma_)
            k_rmse_tr += loss_tr#np.sqrt(2*loss_tr)
            k_rmse_te += loss_te#np.sqrt(2*loss_te)
        rmse_tr.append(k_rmse_tr/k_fold)
        rmse_te.append(k_rmse_te/k_fold)

    cross_validation_visualization(gamma,rmse_tr, rmse_te)
    return rmse_tr, rmse_te
rt, re = cross_valid()

1e-05
fold 0


In [40]:
gamma = np.logspace(-5,-3.7,30)[np.argmin(rt)]

In [50]:
w,e = reg_logistic_regression(yb,std_input,0.4, np.ones(std_input.shape[1]).T, 1000000, gamma )

In [67]:
w,e = least_squares(yb, std_input)

In [47]:
lambdas = np.logspace(-5,0,40)

rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    w,e = ridge_regression(yb, std_input, lambda_)
    rmse_tr.append(e)
#cross_validation_visualization(lambdas, rmse_tr, rmse_te)

In [53]:
w

array([ 0.06482045, -3.30781233, -0.57347127,  2.44345568,  2.36440447,
        0.82069243, -0.06576547,  1.67040561, -1.3048738 ,  3.10764163,
        3.10815649, -0.04477108,  0.33038083,  0.37394302, -0.03598677,
        0.49157133,  0.94885366,  0.35342434,  1.34684554,  1.26126858,
        1.50425369,  0.2695489 ,  0.75178622,  1.45028381])

correlation between features to remove linearly dependant features

In [36]:
features = np.hsplit(input_data, input_data.shape[1])
features_bis = features.copy()
for f1 in features:
    for f2 in features_bis:

SyntaxError: unexpected EOF while parsing (<ipython-input-36-9732a39096c7>, line 4)

In [ ]:
np.corrcoef(std_input).shape

The output data is unbalanced to we should double the error when we detect a -1 instead of a 1

In [ ]:
plt.hist(yb)
plt.show()

In [ ]:
#replace the value -999 by the mean of the feature corresponding to s or b
def replace_nan(splitarray):
    a_shape = splitarray.shape
    splitarray = splitarray.reshape(a_shape[0], -1)
    a_hsplit = np.hsplit(splitarray, a_shape[2])
    for a in a_hsplit:
        temp = a[a>-999]
        mean = np.mean(temp)
        np.place(a, a<=-999, mean)
    return np.hstack(a_hsplit)

    

In [ ]:
def correlation_feat_y(y , feature):
    return np.corrcoef(feature, y)

In [60]:
indexes = np.unique(np.argwhere(input_data<= -999)[:,0])
data_fullrows = np.delete(input_data,np.unique(np.argwhere(input_data<= -999)[:,0]), axis = 0)
y_fullrows = np.delete(yb,indexes,axis = 0)
w, e = least_squares(y_fullrows, data_fullrows)
initial_w = w
print(e)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less_equal
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less_equal
  


ValueError: On entry to DLASCL parameter number 5 had an illegal value

In [ ]:
#calculate the Pearson product-moment correlation coefficients between the output and the features
# WARNING # We should erase the drop the NaN value to compute the correlation. BE CAREFUL to drop the relevant
# indices of y also
correlation_matrices =[]
for feature in np.hsplit(input_data, input_data.shape[1]):
    correlation_matrices.append(correlation_feat_y(yb, feature))
    
#input_data[input_data > -999]
#get the data with prediction s with nan value replaced by the mean of the split feature
s_predict = input_data[np.argwhere(yb>0)]
s_predict = replace_nan(s_predict)
#get the data with prediction b with nan value replaced by the mean of the split feature
b_predict = input_data[np.argwhere(yb<0)]
b_predict = replace_nan(b_predict)
    

    

In [ ]:
#plot the histogram of each features
for i in range(input_data.shape[1]):
    plt.hist(input_data[:,i])
    plt.title('Column: '+ str(i))
    plt.show()

In [57]:
#create a copy of the training inpute date and removes the features with a majority of -999
cop = input_data.copy()
cop = np.delete(cop, [4,5,6,12,23,24,26,27,28],axis= 1)
#initial_w = np.delete(initial_w,[4,5,6,12,23,24,26,27,28] )
cop.shape

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  This is separate from the ipykernel package so we can avoid doing imports until


(250000, 19)

In [58]:
#hsplit_ = np.hsplit(input_data,len(input_data[0]))
print(len(cop[0]))
print(len(initial_w))

19


NameError: name 'initial_w' is not defined

In [59]:
#least squares with the normal equation
#w, e = least_squares(yb, cop)
print(e)
#least squares GD
#w, e = least_squares_SDG(yb, cop, initial_w, 10, 0.2)
e,w = logistic_regression(yb, std_input, initial_w, 10000, 0.01)
print(e)
lambdas = np.logspace(-20,0,50)
ee = 10000
lam = 0
for ind, lambda_ in enumerate(lambdas):
    w, e = ridge_regression(yb, cop, lambda_)
    if e< ee:
        ee = e
        lam = lambda_
print(ee, lam)
#gradient=compute_gradient_mse(yb,cop, w)


#transpose the vector w to use it in the newtons method
#w = w.reshape(w.shape[0],1)

###
# not working yet
###
#loss, w = new_logistic_regression(yb, cop, 0.1, w , 10,0.6)
#loss


142603.98592


NameError: name 'initial_w' is not defined

In [ ]:
e = yb-cop@w
e.T@e/2*len(yb)

In [68]:
#helper function that create the prediction of y for the test data
y_pred = predict_labels(w, std_input_te)


In [69]:
#create the csv file to submit on kraggle
create_csv_submission(ids_te, y_pred, 'least_square_first_test.csv')

In [56]:
len(y_pred)

568238